# Region & Lat/Lon Addition

Code to add world regions and lat long to our final stack event dataset

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
# Open files

events = pd.read_csv('/home/cascade/projects/data_out_urbanheat/All_data20191019.csv')
ghs = gpd.read_file('/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')
countries = pd.read_csv('/home/cascade/tana-crunch-cascade/projects/UrbanHeat/data/raw/countrylist.csv')

In [3]:
events.head()

,Unnamed: 0,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year
0,0,1,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975],1983
1,1,2,2372,France,1,1,43.331635,2.731635,2.731635,['1983.07.31'],[2.731635],[43.331635],1983
2,2,3,6156,Kazakhstan,1,1,41.336376,0.736376,0.736376,['1983.07.31'],[0.736376],[41.336376],1983
3,3,4,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318],1983
4,4,5,2885,Austria,1,1,44.013890,3.413890,3.413890,['1983.07.27'],[3.41389],[44.01389],1983


In [4]:
countries[countries['name'] == 'Serbia']

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
197,Serbia,RS,SRB,688.0,ISO 3166-2:RS,Europe,Southern Europe,Southern Europe,150.0,39.0,NaN


In [5]:
# Fix Ivory Coast
events.CTR_MN_NM = events.CTR_MN_NM.replace('CÃ´te d\'Ivoire', 'Ivory Coast')

## Try Merging Events and Countries

In [6]:
regions = pd.DataFrame()
regions['CTR_MN_NM'] = countries['name']
regions['region'] = countries['region']
regions['sub-region'] = countries['sub-region']
regions['intermediate-region'] = countries['intermediate-region']


In [7]:
print(len(events))
events = events.merge(regions, on = 'CTR_MN_NM', how = 'inner')
print(len(events))

883026
883026


In [8]:
events.head()

,Unnamed: 0,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year,region,sub-region,intermediate-region
0,0,1,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975],1983,Europe,Western Europe,Western Europe
1,3,4,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318],1983,Europe,Western Europe,Western Europe
2,5,22703,2833,Germany,1,1,41.787350,1.187350,1.187350,['1984.07.11'],[1.18735],[41.78735],1984,Europe,Western Europe,Western Europe
3,6,349979,2833,Germany,1,1,41.393425,0.793425,0.793425,['1998.08.11'],[0.793425],[41.393425],1998,Europe,Western Europe,Western Europe
4,0,429537,2833,Germany,1,1,41.278350,0.678350,0.678350,['2001.08.26'],[0.67835],[41.27835],2001,Europe,Western Europe,Western Europe


## Match lat/lon with events

In [9]:
events.shape

(883026, 16)

In [10]:
ghs.head()

df = pd.DataFrame()
df['ID_HDC_G0'] = ghs['ID_HDC_G0']
df['GCPNT_LAT'] = ghs['GCPNT_LAT']
df['GCPNT_LON'] = ghs['GCPNT_LON']

events = events.merge(df, on = 'ID_HDC_G0', how = 'inner')

In [11]:
events.shape

(883026, 18)

In [12]:
events.head()

,Unnamed: 0,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year,region,sub-region,intermediate-region,GCPNT_LAT,GCPNT_LON
0,0,1,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975],1983,Europe,Western Europe,Western Europe,48.138002,11.563435
1,2,763005,2784,Germany,1,1,40.658337,0.058337,0.058337,['2013.08.06'],[0.058337],[40.658337],2013,Europe,Western Europe,Western Europe,48.138002,11.563435
2,3,4,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318],1983,Europe,Western Europe,Western Europe,47.849642,12.102591
3,5,22703,2833,Germany,1,1,41.787350,1.187350,1.187350,['1984.07.11'],[1.18735],[41.78735],1984,Europe,Western Europe,Western Europe,47.849642,12.102591
4,6,349979,2833,Germany,1,1,41.393425,0.793425,0.793425,['1998.08.11'],[0.793425],[41.393425],1998,Europe,Western Europe,Western Europe,47.849642,12.102591


In [13]:
# events.to_csv('/home/cascade/projects/data_out_urbanheat/All_data20191107.csv')

# OLD CODE

In [ ]:
# Rename countries

countries.name = countries.name.replace('Bolivia (Plurinational State of)', 'Bolivia')
countries.name = countries.name.replace('Côte d\'Ivoire', 'CÃ´te d\'Ivoire')
countries.name = countries.name.replace('Iran (Islamic Republic of)','Iran')
countries.name = countries.name.replace('Tanzania, United Republic of','Tanzania')
countries.name = countries.name.replace('United States of America','United States')
countries.name = countries.name.replace('Venezuela (Bolivarian Republic of)','United States')
countries.name = countries.name.replace('Korea (Democratic People\'s Republic of)','North Korea')
countries.name = countries.name.replace('Korea, Republic of','South Korea')
countries.name = countries.name.replace('Lao People\'s Democratic Republic','Laos')
countries.name = countries.name.replace('Syrian Arab Republic','Syria')
countries.name = countries.name.replace('Palestine, State of','Palestina')
countries.name = countries.name.replace('Congo, Democratic Republic of the','Democratic Republic of the Congo')
countries.name = countries.name.replace('Russian Federation','Russia')
countries.name = countries.name.replace('Congo','Republic of Congo')


# {'Kosovo', 'Macedonia', 'Swaziland', 'Venezuela', 'Vietnam'}
# Kosovo, Europe, Southern Europe, NaN
# Macedonia, Europe, Southern Europe, NaN
# Swaziland, Africa, Sub-Saharan, Africa, Southern Africa
# Venezuela Americas, Latin America and the Caribbean, South America
# Vietnam, Asia, South-eastern Asia, NaN


In [ ]:
# Get Regions

regions = pd.DataFrame()
regions['CTR_MN_NM'] = countries['name']
regions['region'] = countries['region']
regions['sub-region'] = countries['sub-region']
regions['intermediate-region'] = countries['intermediate-region']

ghs_cnt = pd.DataFrame()
ghs_cnt['ID_HDC_G0'] = ghs['ID_HDC_G0']
ghs_cnt['CTR_MN_NM'] = ghs['CTR_MN_NM']

In [ ]:
print(ghs_cnt.shape)
merge = ghs_cnt.merge(regions, on = 'CTR_MN_NM', how = 'left')

print(merge.shape)

In [ ]:
merge = ghs_cnt

## Match lat/lon with events

In [ ]:
events.head()

In [ ]:
events.shape

In [ ]:
ghs.head()

df = pd.DataFrame()
df['ID_HDC_G0'] = ghs['ID_HDC_G0']
df['GCPNT_LAT'] = ghs['GCPNT_LAT']
df['GCPNT_LON'] = ghs['GCPNT_LON']

events = events.merge(df, on = 'ID_HDC_G0', how = 'inner')

In [ ]:
events.shape

In [ ]:
## Match countries and regions

In [ ]:
ghs_names = list(np.unique(ghs.CTR_MN_NM))

In [ ]:
cnt_names = list(np.unique(countries.name))

In [ ]:
# reset country names
cnt_names

In [ ]:
c = set(ghs_names) & set(cnt_names)

In [ ]:
print(len(c))
print(len(ghs_names))

In [ ]:
missing = set(ghs_names) - (c)
missing

In [ ]:
# Rename countries

countries[countries['name'] == 'Bolivia (Plurinational State of)'] = 'Bolivia'
countries[countries['name'] == 'Côte d\'Ivoire'] = 'CÃ´te d\'Ivoire'
countries[countries['name'] == 'Iran (Islamic Republic of)'] = 'Iran'
countries[countries['name'] == 'Tanzania, United Republic of'] = 'Tanzania'
countries[countries['name'] == 'United States of America'] = 'United States'
countries[countries['name'] == 'Venezuela (Bolivarian Republic of)'] = 'United States'
countries[countries['name'] == 'Korea (Democratic People\'s Republic of)'] = 'North Korea'
countries[countries['name'] == 'Korea, Republic of'] = 'South Korea'
countries[countries['name'] == 'Lao People\'s Democratic Republic'] = 'Laos'
countries[countries['name'] == 'Syrian Arab Republic'] = 'Syria'
countries[countries['name'] == 'Palestine, State of'] = 'Palestina'
countries[countries['name'] == 'Congo, Democratic Republic of the'] = 'Democratic Republic of the Congo'
countries[countries['name'] == 'Russian Federation'] = 'Russia'
countries[countries['name'] == 'Congo'] = 'Republic of Congo'


# {'Kosovo', 'Macedonia', 'Swaziland', 'Venezuela', 'Vietnam'}

In [ ]:
countries

In [ ]:
# Add region column

# events['region'] = 'NAN'

# for i, event in events.iterrows():
#     for j, country in countries.iterrows():
#         if event['CTR_MN_NM'] == country['name']:
#             if country['intermediate-region'] == 'NaN':
#                 event['region'] == country['sub-region']
#             else:
#                 event['region'] == country['intermediate-region']

regions = pd.DataFrame()

regions['CTR_MN_NM'] = countries['name']
regions['region'] = countries['region']
regions['sub-region'] = countries['sub-region']
regions['intermediate-region'] = countries['intermediate-region']

regions

In [ ]:
sub = events[0:1000]
merge = sub.merge(regions, on = 'CTR_MN_NM', how = 'inner')

In [ ]:
merge.tail()